#### 1. Import pandas library

In [51]:
import pandas as pd

#### 2. Import users table:

In [52]:
users = pd.read_csv('users_table.csv')
users.head(3)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN


#### 3. Rename Id column to userId

In [53]:
users.rename(columns={'Id':'userId'}, inplace=True)
users.columns

Index(['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl'],
      dtype='object')

#### 4. Import posts table:

In [54]:
posts = pd.read_csv('posts_table.csv')
posts.head(3)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [55]:
posts.rename(columns={'Id':'postId', 'OwnerUserId': 'userId'}, inplace=True)
posts.columns

Index(['postId', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score',
       'ViewCount', 'Body', 'userId', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName'],
      dtype='object')

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [56]:
newusers = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
newposts = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]
newusers

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
...,...,...,...,...,...
40320,55743,1,0,0,0
40321,55744,6,1,0,0
40322,55745,101,0,0,0
40323,55746,106,1,0,0


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [57]:
merged_df = pd.merge(newusers, newposts, left_index = True, right_index = True)
merged_df

,userId_x,Reputation,Views,UpVotes,DownVotes,postId,Score,userId_y,ViewCount,CommentCount
0,-1,1,0,5007,1920,1,23,8.0,1278.0,1
1,2,101,25,3,0,2,22,24.0,8198.0,1
2,3,101,22,19,0,3,54,18.0,3613.0,4
3,4,101,11,0,0,4,13,23.0,5224.0,2
4,5,6792,1145,662,5,5,81,23.0,NaN,3
...,...,...,...,...,...,...,...,...,...,...
39995,55412,6,1,0,0,48321,0,19966.0,NaN,0
39996,55414,1,0,0,0,48322,3,892.0,NaN,2
39997,55415,6,2,0,0,48323,1,2020.0,NaN,0
39998,55416,111,0,0,0,48324,3,19914.0,NaN,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [58]:
merged_df.isnull().sum()

userId_x            0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
userId_y         1038
ViewCount       24105
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [59]:
#finding percentage of null values in both columns
print(merged_df['userId_y'].isnull().sum() * 100/ len(merged_df))
print(merged_df['ViewCount'].isnull().sum() * 100/ len(merged_df))

2.595
60.2625


In [48]:
merged_df[merged_df.isna().any(axis=1)].head(10)

,userId_x,Reputation,Views,UpVotes,DownVotes,postId,Score,userId_y,ViewCount,CommentCount
4,5,6792,1145,662,5,5,81,23.0,NaN,3
8,10,121,20,2,0,9,13,50.0,NaN,3
11,13,817,178,44,1,12,20,5.0,NaN,1
12,15,11,8,0,0,13,14,23.0,NaN,4
13,16,101,4,7,0,14,5,36.0,NaN,1
14,17,345,14,13,1,15,13,6.0,NaN,0
15,18,128,8,16,0,16,16,8.0,NaN,3
16,19,101,13,2,0,17,9,NaN,1261.0,0
17,21,101,6,0,0,18,31,36.0,NaN,1
18,22,591,90,693,6,19,12,55.0,NaN,0


In [ ]:
#remove the ViewCount series because even though it has more than 50% of nan Values thus not providing sufficient info 
#leave the userId_y series because only a tiny percentage has missing values

In [63]:
merged_df.drop(['ViewCount'], axis=1, inplace=True)
merged_df

,userId_x,Reputation,Views,UpVotes,DownVotes,postId,Score,userId_y,CommentCount
0,-1,1,0,5007,1920,1,23,8.0,1
1,2,101,25,3,0,2,22,24.0,1
2,3,101,22,19,0,3,54,18.0,4
3,4,101,11,0,0,4,13,23.0,2
4,5,6792,1145,662,5,5,81,23.0,3
...,...,...,...,...,...,...,...,...,...
39995,55412,6,1,0,0,48321,0,19966.0,0
39996,55414,1,0,0,0,48322,3,892.0,2
39997,55415,6,2,0,0,48323,1,2020.0,0
39998,55416,111,0,0,0,48324,3,19914.0,0


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [64]:
merged_df.dtypes

userId_x          int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
userId_y        float64
CommentCount      int64
dtype: object

In [68]:
merged_df['userId_y'] = merged_df['userId_y'].fillna(0)
merged_df['userId_y'] = merged_df['userId_y'].astype('int64')

In [70]:
merged_df.dtypes

userId_x        int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
userId_y        int64
CommentCount    int64
dtype: object